<a href="https://colab.research.google.com/github/zakihir0/jigsaw---Agile-Community-Rules-Classification/blob/main/Jigsaw_%7C_Llama3_1_8B_Instruct_Training_one_epoch_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

jigsaw_agile_community_rules_path = kagglehub.competition_download('jigsaw-agile-community-rules')

print('Data source import complete.')


100%|██████████| 695k/695k [00:00<00:00, 16.4MB/s]

Extracting files...
Data source import complete.


In [ ]:
!pip install -U unsloth
!pip install -U torchvision

In [ ]:
from unsloth import FastLanguageModel
import torch
import os
from datasets import load_dataset, Dataset
from peft import get_peft_model, LoraConfig, prepare_model_for_kbit_training
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, LlamaForSequenceClassification
from trl import SFTConfig, SFTTrainer
import pandas as pd
import numpy as np

max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 32,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

seed = 52
device = "cuda" if torch.cuda.is_available() else "cpu"

prompt = '''You are given a comment on reddit and a rule. Your task is to classify whether the comment violates the rule. Only respond Yes/No.'''

if os.path.exists("/kaggle/input"):
  train_df = pd.read_csv('/kaggle/input/jigsaw-agile-community-rules/train.csv')
  test_df = pd.read_csv('/kaggle/input/jigsaw-agile-community-rules/test.csv')

else:
  train_df = pd.read_csv(os.path.join(jigsaw_agile_community_rules_path, "train.csv"))
  test_df = pd.read_csv(os.path.join(jigsaw_agile_community_rules_path, "test.csv"))

from datasets import load_dataset, Dataset
from datasets import load_dataset
from unsloth.chat_templates import get_chat_template
from unsloth.chat_templates import standardize_sharegpt

# tokenizer = get_chat_template(
#     tokenizer,
#     chat_template = "qwen-2.5",
# )
user_prompt = """
Subreddit: r/{subreddit}
Rule: {rule}
Examples:
1) {positive_example_1}
Violation: Yes

2) {negative_example_1}
Violation: No

3) {negative_example_2}
Violation: No

4) {positive_example_2}
Violation: Yes
Comment:
{body}
Violation: """

classes = ['No', 'Yes']

columns = ['subreddit', 'rule', 'positive_example_1', 'positive_example_2', 'negative_example_1', 'negative_example_2', 'body', 'rule_violation']
dataset = [
    [{"role": "system", "content": prompt},
    {"role": "user", "content": user_prompt.format(subreddit=subreddit,
                                                   rule=rule,
                                                   positive_example_1=positive_example_1,
                                                   positive_example_2=positive_example_2,
                                                   negative_example_1=negative_example_1,
                                                   negative_example_2=negative_example_2,
                                                   body=body)},
    {"role": "assistant", "content": classes[target]}]
    for subreddit, rule, positive_example_1, positive_example_2, negative_example_1, negative_example_2, body, target  in train_df[columns].values]

def formatting(dataset):
    texts = []
    for i in range(len(dataset)):
        texts.append(tokenizer.apply_chat_template(dataset[i], tokenize=False, add_generation_prompt=False))
    return Dataset.from_dict({'text': texts})

dataset = formatting(dataset)

from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 4,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # max_steps = 5,
        num_train_epochs = 1,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 10,
        optim = "paged_adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)

from unsloth.chat_templates import train_on_responses_only

trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>",
    response_part = "<|start_header_id|>assistant<|end_header_id|>",
)

trainer_stats = trainer.train()

import shutil

for path in os.listdir('/kaggle/working'):
    try:
        shutil.rmtree('/kaggle/working/' + path)
    except:
        pass

model.save_pretrained_merged("llama-8b-instruct-jigsaw", tokenizer)

==((====))==  Unsloth 2025.10.3: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/2029 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/2029 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,029 | Num Epochs = 1 | Total steps = 508
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 4 x 1) = 4
 "-____-"     Trainable parameters = 83,886,080 of 8,114,147,328 (1.03% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,0.823400
20,0.271300
30,0.227800
40,0.632800
50,0.270900
60,0.294500
70,0.216700
80,0.193400
90,0.331200
100,0.228700


Step,Training Loss
10,0.823400
20,0.271300
30,0.227800
40,0.632800
50,0.270900
60,0.294500
70,0.216700
80,0.193400
90,0.331200
100,0.228700
